In [8]:
import psycopg2
from dotenv import load_dotenv
import os
from psycopg2 import extras
import json

load_dotenv()

# Connection parameters
dbname = os.environ['DATABASE_NAME']
user = os.environ['DATABASE_USER']
password = os.environ['DATABASE_PASSWORD']
host = 'localhost'
port = 5444

# Establishing the connection
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Connected to database successfully!")
except psycopg2.Error as e:
    print("Unable to connect to the database:", e)


# Define a function to create the table if it does not exist
def create_table_if_not_exists(conn, table_name):
    cursor = conn.cursor()
    try:
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id SERIAL PRIMARY KEY, data JSONB)")
        conn.commit()
        print("Table created successfully or already exists!")
    except psycopg2.Error as e:
        print("Error creating table:", e)
    finally:
        cursor.close()

# Define a function to insert JSON data into the database
def insert_json_data(conn, table_name, json_data):
    cursor = conn.cursor()
    try:
        cursor.execute(f"INSERT INTO {table_name} (data) VALUES (%s)", (json.dumps(json_data),))
        conn.commit()
        print("Inserted JSON data into the table successfully!")
    except psycopg2.Error as e:
        print("Error inserting JSON data into the table:", e)
    finally:
        cursor.close()

# Define the table name
table_name = 'test_json'

# Create the table if it does not exist
create_table_if_not_exists(conn, table_name)

# Get the list of JSON files
json_files_dir = '../data/ntb_json/'
json_files = os.listdir(json_files_dir)[:2]  # Assuming you want to process the first two files

# Iterate over the JSON files and insert their data into the database
for json_file in json_files:
    with open(os.path.join(json_files_dir, json_file), 'r') as f:
        json_data = json.load(f)
        insert_json_data(conn, table_name, json_data)

# Close the database connection
conn.close()
print("Connection closed.")

Connected to database successfully!
Table created successfully or already exists!
Inserted JSON data into the table successfully!
Inserted JSON data into the table successfully!
Connection closed.
